In [1]:
#!pip install --upgrade pika

In [2]:
import pika
pika.__version__

'1.1.0'

In [3]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [4]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [5]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

In [6]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

In [7]:
def on_request(ch, method, props, body):
    n = int(body) # body is the message to the worker. We want to calculate Fib(n)

    print(" I need to calculate fib(%s)" % n)
    
    print('ch:', ch)
    print('method:', method)
    print('props:', props)
    
    print('props.reply_to:', props.reply_to)
    print('props.correlation_id:', props.correlation_id)
    
    response = fib(n) # Results, Fib(n), calculated
    
    # We need to send back the results to the client
    ch.basic_publish(exchange='', # Default exchange
                     routing_key=props.reply_to, # This is the routing key to the queue that we use for sending the answer
                     #use the same correlation_id so the client knows that this is a reply to what message
                     properties=pika.BasicProperties(correlation_id = props.correlation_id), 
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)
    print(" Fib(%s) Calculation done. Results sent back to the server" % n)
    print(" *** \n")


In [8]:
channel.queue_declare(queue='rpc_queue')
channel.basic_qos(prefetch_count=1)

In [9]:
#When the worker receives a message it will tun the "on_request" call back function
channel.basic_consume(queue='rpc_queue', on_message_callback=on_request)  

'ctag1.12f39371161b4cb381a9df14469d5115'

In [ ]:
print(" [x] Awaiting RPC requests")
channel.start_consuming()

 [x] Awaiting RPC requests
 I need to calculate fib(30)
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7f931c219c18> params=<ConnectionParameters host=PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com port=5672 virtual_host=/ ssl=False>>>>
method: <Basic.Deliver(['consumer_tag=ctag1.12f39371161b4cb381a9df14469d5115', 'delivery_tag=1', 'exchange=', 'redelivered=False', 'routing_key=rpc_queue'])>
props: <BasicProperties(['correlation_id=8ae873a0-ccad-4750-849e-423f2d4be8d8', 'reply_to=amq.gen-Iwf8D-LXL713ChiKbhEBpQ'])>
props.reply_to: amq.gen-Iwf8D-LXL713ChiKbhEBpQ
props.correlation_id: 8ae873a0-ccad-4750-849e-423f2d4be8d8
 Fib(30) Calculation done. Results sent back to the server
 *** 

 I need to calculate fib(20)
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport objec